![Embeddings](./images/AIStylist.png)
# AI Stylist - Conversational Shopping Experience with Amazon Bedrock


In this notebook, we will build an AI virtual assistant acting as your stylist. You will learn how to build an application that help users choose clothes from a product catalog, all powered by Large Language Models (LLMs). We will leverage Amazon Bedrock for using the Foundation Models (FMs).

### Context

Virtual Assistants are becoming increasingly popular as LLMs start to power the interaction with users. These applications demand an ability to `understand` the intent of the user input and generate adaptable sequence of calls to language models and various utilities depending on user input.

### Overview

Conversational interfaces such as chatbots and virtual assistants can be used to enhance the user experience of the customers. Virtual assistants use natural language processing (NLP) and machine learning algorithms to understand and respond to user queries. Chatbots can be used in a variety of applications, such as customer service, sales, and e-commerce, to provide quick and efficient responses to users. They can be accessed through various channels such as websites, social media platforms, and messaging apps.

### Pattern

We can improve upon this process by implementing an architecture called `Retreival Augmented Generation (RAG)`. RAG retrieves data from outside the language model (non-parametric) and augments the prompts by adding the relevant retrieved data in context. Virtual assistants need ability to determine the sequence of events needed to generate the final results and as such we build towards the use, `plan-and-execute` agents along with `Zero-shot ReAct` which is an action agent and uses [`ReAct`](https://arxiv.org/pdf/2205.00445.pdf) pattern to select the appropriate calls. These will then develop into `Agents` concept. The Agent interface enables such flexibility for these applications.

There are two primary categories of agents:

- Action agents: At each interval, determine the subsequent action utilizing the outputs of all previous actions. 
- Plan-and-execute agents: Determine the complete order of actions initially, then implement them all without updating the plan.

### Challenges
- Parse the user query and extracting information which can be used to search the catalog
- Reform the prompts to bring products from the catalog
- Search product catalog to bring relevant products with details like product id, description etc
- Generate relevant images for a style based on the user chat
- Continue the shopping experience workflow for adding products or finalizing the sale
- Identify relevant products based on user location or questions asked by user

### Proposal

To the above challenges, this notebook proposes the following strategy

#### Prepare documents

![Embeddings](./images/embeddings_lang.png)

Before being able to answer the questions, the product catalog must be processed and a stored in a document store index
- Load the product catalog and specifically the descriptions
- Process and split them into smaller chunks
- Create a numerical vector representation of each chunk using Amazon Bedrock Titan Embeddings model
- Create an index using the chunks and the corresponding embeddings

#### Catalog operations. Fetch relevant products from the catalog

We need to leverage the LLM for the following 
- Extract information of the `intent` and other details like `place` , objective of the purchase
- Use [LangChain](https://python.langchain.com/docs/get_started/introduction) as orchestrator to do a similarity search from the vector store and bring back the artifacts matching the query
- Extract information for `Product ID` from the search results  and conduct a search to bring further details of the selected products
- Leverage [LangChain](https://python.langchain.com/docs/get_started/introduction) to orchestrate and call API's using the `ReAct` framework to bring other details like weather and conduct search to bring back the relevant products

#### Query

- Identify the `intent` of the query
- Orchestrate the workflow to move the users to the next steps in their purchase journey

![Question](./images/chatbot_lang.png)

When the document's index is prepared, you are ready to ask the questions and relevant documents will be fetched based on the question being asked. Following steps will be executed.
- Create an embedding of the input question
- Compare the question embedding with the embeddings in the index
- Fetch the (top N) relevant document chunks
- Add those chunks as part of the context in the prompt
- Send the prompt to the model under Amazon Bedrock
- Get the contextual answer based on the documents retrieved


## Virtual Assistant using Amazon Bedrock

![Amazon Bedrock - Conversational Interface](./images/context-aware-chatbot.png)

## Langchain framework for building Virtual Assistants with Amazon Bedrock
In Conversational interfaces such as virtual assistants, it is highly important to remember previous interactions, both at a short term but also at a long term level.

LangChain provides memory components in two forms. First, LangChain provides helper utilities for managing and manipulating previous chat messages. These are designed to be modular and useful regardless of how they are used. Secondly, LangChain provides easy ways to incorporate these utilities into chains.
It allows us to easily define and interact with different types of abstractions, which make it easy to build powerful chatbots.

## Building Chatbot with Context - Key Elements

The first process in a building a contextual-aware chatbot is to **generate embeddings** for the context. Typically, you will have an ingestion process which will run through your embedding model and generate the embeddings which will be stored in a sort of a vector store. In this example we are using Titan Embeddings model for this

![Embeddings](./images/embeddings_lang.png)

Second process is the user request orchestration , interaction,  invoking and returning the results

![Chatbot](./images/chatbot_lang.png)

## Architecture
![Architecture](./images/Architecture.png)

### <font color='red'>Setup</font> 
---
<font color='red'>⚠️ ⚠️ ⚠️</font> 
Before running this notebook, ensure you've run the [Intro to Bedrock notebook](./intro_to_bedrock.ipynb) notebook. <font color='red'>⚠️ ⚠️ ⚠️</font>

---

## Configure Bedrock

Create the necessary clients to invoke Bedrock models. If you would need to pass in a certain role then set those values appropriately
We begin with instantiating the LLM and the Embeddings model. Here we are using Anthropic Claude V2 for text generation and Titan Embeddings G1 - Text for text embeddings.

Note: It is possible to choose other models available with Bedrock. You can replace the `model_id` as follows to change the model.

`llm = Bedrock(model_id="anthropic.claude-v2")`

You can read about all the available model IDs [here](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import json
import os
import sys
import boto3
import botocore

from langchain.llms.bedrock import Bedrock
from IPython.display import Image

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

os.environ["AWS_DEFAULT_REGION"] = "us-west-2"
# os.environ["AWS_PROFILE"] = ""
# os.environ["BEDROCK_ASSUME_ROLE"] = ""  # E.g. "arn:aws:..."

# admin client
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=False)

# model execution client
bedrock_runtime = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None))

model_parameter = {
    "temperature": 0.0,
    "top_p": .5,
    "top_k": 250,
    "max_tokens_to_sample": 2000,
    "stop_sequences": ["\n\n Human: bye"]
}

# configure the LLM for reuse
llm = Bedrock(
    model_id="anthropic.claude-v2",
    model_kwargs=model_parameter,
    client=bedrock_runtime
)

## Implementation
In order to follow the approach this notebook is using the LangChain framework where it has integrations with different services and tools that allow efficient building of patterns such as RAG. We will be using the following tools:

- **LLM (Large Language Model)**: Anthropic Claude V2 available through Amazon Bedrock

  This model will be used to understand the document chunks and provide an answer in human friendly manner.
- **Embeddings Model**: Amazon Titan Embeddings available through Amazon Bedrock

  This model will be used to generate a numerical representation of the textual documents
- **Document Loader**: [S3FileLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain.document_loaders.s3_file.S3FileLoader.html) and PDF Loader available through LangChain

  This is the loader that can load the documents from a source, for the sake of this notebook we are loading the sample files from a local path. This could easily be replaced with a loader to load documents from enterprise internal systems.

- **Vector Store**: In-Memory store FAISS

  The index helps to compare the input embedding and the document embeddings to find relevant document
- **Wrapper**: wraps index, vector store, embeddings model and the LLM to abstract away the logic from the user.

## Simulate a user ask of the AI Stylist 

![Generate Look](./images/user_prompt.png)

### Try these prompts if you would like
- `"I am a female journalist in my 30s traveling to New York next week. What kind of outfit should I wear on my first day at New York Times?"`
- `"I am male athlete looking for suitable outfit for attending half marathon in San Fransisco."`

In [ ]:
# If you'd like to try your own prompt, edit this parameter!
# customer_input = (
#     "I am a male consultant in my 30s traveling to New York next week. "
#     "What kind of outfit should I wear on my first day in the office?"
# )

customer_input = (
    "I am a young woman in early 20's traveling to Norway in January. "
    "What kind of outfit should I wear to view the northern lights north "
    "of the arctic circle?"
)


# Customer id to infuse order history and delivery address
customer_id = "2"

## Extract `product relevant` information

We have our product catalog stored in a data base and the key attributes for those are 
1. Product type
2. Age group for the product
3. Season for use of the product
4. Description of the product
5. Images for the product

We will instruct LLM to fetch the relevant information from the user prompt based on the above so we can query our catalog and bring back relevant results. Note we have a very specific prompt template for the `entity extraction`

In [ ]:
# Identify product attributes from customer prompt to generate better results

# The prefix "ner" stands for Named Entity Recognition. It is a subtask of
# information extraction in natural language processing (NLP) that involves
# identifying and classifying named entities mentioned in text into predefined
# categories such as the names of persons, organizations, locations, dates,
# and other entities.

# For example, in the sentence "Barack Obama was born in Hawaii," NER would
# identify "Barack Obama" as a person and "Hawaii" as a location.

ner_prompt = """Human: Find person age group, gender, season and the location in the customer input.
Instructions:
The age group can be one of the following: 10-20, 20-30, 30-50, 50+
The gender can be one of the following: Mens, Womens, Other
The gender can also be derived from the name if not explicitly mentioned
The season can be one of the following: summer, winter, spring, fall
The output must be in JSON format inside the tags <attributes></attributes>
Today's date is June 1st.

If the information of an entity is not available in the input then don't include that entity in the JSON output

Begin!

Customer input: {customer_input}
Assistant:"""
entity_extraction_result = llm(
    ner_prompt.format(customer_input=customer_input)).strip()
print(entity_extraction_result)

#### Extract values into JSON

Since we have instructed LLM to return our data as XML wrapping a JSON, we run the necessary extraction steps to fetch the relevant details.

In [ ]:
import re
import json

result = re.search(
    '<attributes>(.*)</attributes>',
    entity_extraction_result,
    re.DOTALL)
attributes = json.loads(result.group(1))
attributes

## Use Retrieval Augmented Generation (RAG) 

We will leverage the semantic search to embed product-accessory catalogs and order history for the embeddings created by `Amazon Titan Embeddings Text v1`

After downloading we can load the documents with the help of [S3FileLoader available under LangChain](https://python.langchain.com/docs/modules/data_connection/document_loaders/) and splitting them into smaller chunks.

Note: The retrieved document/text should be large enough to contain enough information to answer a question; but small enough to fit into the LLM prompt. Also the embeddings model has a limit of the length of input tokens limited to 8k tokens, which roughly translates to ~32000 characters. For the sake of this use-case we are creating chunks of roughly 1000 characters with an overlap of 100 characters using [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html).

Here we are fetching our product catalog and creating the embeddings for 
1. Customer reviews
2. Order History
3. Product catalog description 

In [ ]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import S3FileLoader
import numpy as np

# - create the Titan Embeddings Model
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1",
                                       client=bedrock_runtime)
customer_reviews_loader = S3FileLoader(
    "sagemaker-example-files-prod-us-east-1",
    "datasets/image/howser-bedrock/data/aistylist/data/customer_reviews.csv",
    region_name="us-east-1")
order_history_loader = S3FileLoader(
    "sagemaker-example-files-prod-us-east-1",
    "datasets/image/howser-bedrock/data/aistylist/data/order_history.csv",
    region_name="us-east-1")
products_catalog_loader = S3FileLoader(
    "sagemaker-example-files-prod-us-east-1",
    "datasets/image/howser-bedrock/data/aistylist/data/products_catalog.csv",
    region_name="us-east-1")

documents = []
customer_reviews = customer_reviews_loader.load()
order_history = order_history_loader.load()
products_catalog = products_catalog_loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, to test an alternate behavior.
    chunk_size = 1000,
    chunk_overlap  = 100,
)

# Embedding data here. Add more data for better results
documents += text_splitter.split_documents(customer_reviews)
documents += text_splitter.split_documents(order_history)
documents += text_splitter.split_documents(products_catalog)

vectorstore_faiss = FAISS.from_documents(
    documents, # Documents to embed
    bedrock_embeddings, # Embedding model
)

wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss)

# create an embedding for the customer input which is then used to query the
# index
query_embedding = vectorstore_faiss.embedding_function(customer_input)
np.array(query_embedding)


## Generate *`n`* style recommendations

Make a query to embed the LLM using customer input. Using LangChain for orchestration of RAG. It also provides a framework for orchestrating RAG flows with what purpose built "chains". In this section, we will see how to be a [retrieval chain](https://python.langchain.com/docs/use_cases/question_answering/vector_db_qa) which is more comprehensive and robust than the original retrieval system we built above.

The workflow we used above follows the following process:
1. User input is received.
2. User input is queried against the vector database to retrieve the relevant products
3. Product description and chat memory are inserted into a new prompt to respond to the user input.
4. This output is fed into the stable diffusion model to return the relevant images

However, more complex methods of interacting with the user input can generate more accurate results in RAG architectures. One of the popular mechanisms which can increase accuracy of these retrieval systems is utilizing more than one call to an LLM in order to reformat the user input for more effective search to your vector database. A better workflow is described below compared to the one we already built...

1. User input is received.
2. An LLM is used to reword the user input to be a better search query for the vector database based on the chat history and product description. 
3. This could include things like condensing, rewording, addition of chat context, or stylistic changes.
4. Reformatted user input is queried against the vector database to retrieve relevant products.The reformatted user input and relevant documents are inserted into a new prompt in order to generate the new style. 
5. This is then fed into the stable diffusion model to generate the images. 

In your application the images can come from a pre canned images 

We will now build out this second workflow using LangChain below. First we need to make a prompt which will reformat the user input to be more compatible for searching of the vector database. The way we do this is by providing the chat history as well as the some basic instructions to Claude and asking it to condense the input into a single output. 

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """Human: Use the following pieces of context to generate 5 style recommendations for the customer input at the end.
<context>
{context}
</context>
<example>A navy suit with a light blue dress shirt, conservative tie, black oxford shoes, and a leather belt.</example>
<example>A lehenga choli set with a crop top, flowing skirt, and dupatta scarf in lively colors and metallic accents.</example>

Customer Input: {question}
Each style recommendation must be inside the tags <style></style>.
Do not output product physical IDs.
Skip the preamble.
Assistant: """
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

# Use RetrievalQA customizations for improving Q&A experience
# qa = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=vectorstore_faiss.as_retriever(
#         search_type="similarity", search_kwargs={"k": 6}
#     ),
#     return_source_documents=False,
#     chain_type_kwargs={"prompt": PROMPT},
# )
# styles_response = qa({"query": customer_input})['result']

# Alternatively we can query using wrapper also
styles_response = wrapper_store_faiss.query(question= customer_input, llm=llm)

print_ww(styles_response)

### Prepare the received response

Since we have instructed LLM to return our data is returned as XML wrapping a JSON, we run the necessary extraction steps to fetch the relevant details to generate images for each look. 

In [ ]:
# Prepare input to fetch images for each look
# styles = re.findall('<style>(.*?)</style>', styles_response)

# uncomment when using the wrapper
styles = [
    style[2:]
    for style in styles_response.splitlines()[1:-1]
    if len(style) > 0]
styles

## Generate Images for the relevant style

Generate an image for each look using the `Stable Diffusion` model

![Generate Look](./images/generate_look.png)

In [ ]:
from IPython import display
import json

# Fetching images for each of style
gender_map = {
    'Womens': 'of a female ',
    'Mens': 'of a male '
}

image_strip = ""
for i, style in enumerate(styles):
    gender_str = gender_map.get(attributes.get('gender'))
    request = json.dumps({
        "text_prompts": [
            {
                "text": (
                    f"Full body view {gender_str}"
                    "without a face in " + style + "dslr, ultra quality, "
                    "dof, film grain, Fujifilm XT3, crystal clear, 8K UHD"
                ),
                "weight": 1.0
            },
            {"text": "poorly rendered", "weight": -1.0}
        ],
        "cfg_scale": 9,
        "seed": 4000,
        "steps": 50,
        "style_preset": "photographic",
    })
    model_id = "stability.stable-diffusion-xl-v1"

    response = bedrock_runtime.invoke_model(body=request, modelId=model_id)
    response_body = json.loads(response.get("body").read())

    base_64_img_str = response_body["artifacts"][0].get("base64")

    # this will display the images in a vertical column
    # display.display(display.Image(b64decode(base_64_img_str), width=200))

    # this will display the images in a horizontal strip
    image_strip += (
        "<td><img src='data:image/png;base64, "
        f"{base_64_img_str}"
        "'></td>"
    )

# display the image strip in a HTML table
display.display(display.HTML("<table><tr>" + image_strip +"</tr></table>"))

## Enhance user experience with Chatbot

#### Generating detailed overview based on customer reviews of products in catalog 
We have discussed the key building blocks needed for the chatbot application and now we will start to create them. LangChain's [ConversationBufferMemory](https://python.langchain.com/docs/use_cases/question_answering/chat_vector_db) class provides an easy way to capture conversational memory for LLM chat applications. We will have Claude being able to retrieve context through conversational memory using the prompt template. Note that this time our prompt template includes a {chat_history} variable where our chat history will be included to the prompt.

The prompt template has both conversation memory as well as chat history as inputs along with the human input. Notice how the prompt also instructs Claude to not answer questions which it does not have the context for. This helps reduce hallucinations which is extremely important when creating end user facing applications which need to be factual.


![Architecture](./images/chatbot_products.png)

In [ ]:
chat_prompt1 = "Show me specific reviews that talk about the quality of the fabric for the jacket."
chat_prompt2 = "What do people like about the jacket?"

from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT

print(CONDENSE_QUESTION_PROMPT)

chat_history = [" "]
memory_chain = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True)
conversation = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore_faiss.as_retriever(),
    memory=memory_chain,
    condense_question_prompt=CONDENSE_QUESTION_PROMPT,
    #verbose=True,
    chain_type='stuff', # 'refine',
    #max_tokens_limit=300
)

# Generate detailed reviews based on customer reviews of specific clothing
# in product catalog

try:
    chat_res1 = conversation.run({
        'question': chat_prompt1, 'chat_history': chat_history })
    print_ww(chat_res1)
    chat_history.append([chat_prompt1, chat_res1])
except ValueError as error:
    if  "AccessDeniedException" in str(error):
        class StopExecutionError(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecutionError
    else:
        raise error

In [ ]:
try:

    chat_res2 = conversation.run({
        "question": (
            chat_prompt2
        ),
        "chat_history": chat_history
    })
    print_ww(chat_res2)
    chat_history.append([chat_prompt2, chat_res2])

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        class StopExecutionError(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecutionError
    else:
        raise error

#### Customer order history semantic searches

Generating size and color recommendations based on customer's order history. This will help to provide curated content to the customer

In [ ]:
chat_prompt3 = "What size and color should I wear?"
chat_res3 = wrapper_store_faiss.query(
    question= (
        f"{chat_prompt3} based on order history for customer with id "
        f"{customer_id}"
    ), llm=llm)
print_ww(chat_res3)

## Showing final products based on customer style selection 

Continuing on our architectural pattern we will change the prompt template and leverage the LLM to generate the `recommended` products based on the user selection and weather and other details. The key extraction entities will be 

1. Leverage the customer initial prompt to generate the relevant ids
2. Extract the relevant products from the vector store
3. Physical ID for the products needed



![Architecture](./images/other_products.png)

In [ ]:
prompt_template2 = """Human: Extract list of products and their respective physical IDs from catalog that matches the style given below. 
The catalog of products is provided under <catalog></catalog> tags below.
<catalog>
{context}
</catalog>
Style: {question}

The output should be a JSON of the form <products>[{{"product": <description of the product from the catalog>, "physical_id":<physical id of the product from the catalog>}}, ...]</products>
Skip the preamble.
Assistant: """

PROMPT2 = PromptTemplate(
    template=prompt_template2, input_variables=["context", "question"]
)
qa2 = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_faiss.as_retriever(
        search_type="similarity", search_kwargs={"k": 10}
    ),
    chain_type_kwargs={"prompt": PROMPT2},
    return_source_documents=True,
)

selected_style = styles[3]
print(selected_style)
cart_items = qa2({"query": selected_style })['result']
print_ww(cart_items)

In [ ]:
products = json.loads(re.findall('<products>(.*?)</products>', cart_items, re.DOTALL)[0])
products

In [ ]:
from IPython import display
import urllib.parse

cart_item_strip = ""
for product in products:
    url = "https://sagemaker-example-files-prod-us-east-1.s3.us-east-1.amazonaws.com/datasets/image/howser-bedrock/data/aistylist/images/products/" + urllib.parse.quote(product['physical_id'].strip(), safe='', encoding=None, errors=None) + ".jpg"
    # im = Image.open(requests.get(url, stream=True).raw)
    cart_item_strip += "<td><img src='"+ url + "'></td>"
display.display(display.HTML("<table><tr>" + cart_item_strip +"</tr></table>"))

## Integrating DIY Agents to associate external APIs and databases
### Using ReAct: Synergizing Reasoning and Acting in Language Models Framework
Large language models can generate both explanations for their reasoning and task-specific responses in an alternating fashion. 

Producing reasoning explanations enables the model to infer, monitor, and revise action plans, and even handle unexpected scenarios. The action step allows the model to interface with and obtain information from external sources such as knowledge bases or environments.

The ReAct framework could enable large language models to interact with external tools to obtain additional information that results in more accurate and fact-based responses. Here we will leverage the user prompt and perform the following actions
1. Extract the city 
2. Get weather information
3. Search our product catalog using semantic search to find relevant products
4. Display the products for user to add to cart

![Architecture](./images/weather.png)

In [ ]:
import python_weather

async def getweather(city):
  # declare the client. the measuring unit used defaults to the metric system (celcius, km/h, etc.)
  async with python_weather.Client(unit=python_weather.IMPERIAL) as client:
    # fetch a weather forecast from a city
    weather = await client.get(city)

    # returns the current day's forecast temperature (int)
    return weather

## Accessory recommendations 

We will provide  accessory recommendations based on location provided in customer input

In [ ]:
await getweather(entity_extraction_result[2])

In [ ]:
accessory_response = None
if attributes["location"]:
    current_weather = await getweather(entity_extraction_result[2])
    accessory_input = "Suggest list of accessories based on the weather and the selected style. It is " + current_weather.description + " with temperature at " + str(current_weather.temperature) +" degrees fahrenheit.\n Selected Style: " + styles[0]
    accessory_response = qa({"query": accessory_input})['result']
    print_ww(accessory_response)

In [ ]:
if accessory_response:
    accessories = re.findall('<style>(.*?)</style>', accessory_response)
    accessories_items = qa2({"query": ', '.join(accessories)})['result']
    accessories_items = json.loads(re.findall('<products>(.*?)</products>', accessories_items, re.DOTALL)[0])
    accessory_strip = ""
    for accessory in accessories_items:
        url = "https://sagemaker-example-files-prod-us-east-1.s3.us-east-1.amazonaws.com/datasets/image/howser-bedrock/data/aistylist/images/products/" + urllib.parse.quote(accessory['physical_id'].strip(), safe='', encoding=None, errors=None) + ".jpg"
        accessory_strip += "<td><img src='"+ url + "'></td>"
    display.display(display.HTML("<table><tr>" + accessory_strip +"</tr></table>"))

## Simulate the order check out

Add a customer data table to complete the order transaction. This information provides the shipping address for the outfit order.

In [ ]:
customer_table=[{"id": 1, "first_name": "John", "last_name": "Doe", "age": 35, "address": "123 Bedrock st, California 90210"},
  {"id": 2, "first_name": "Jane", "last_name": "Smith", "age": 27, "address": "234 Sagemaker drive, Texas 12345"},
  {"id": 3, "first_name": "Bob", "last_name": "Jones", "age": 42, "address": "111 DeepRacer ct, Virginia 55555"},
  {"id": 4, "first_name": "Sara", "last_name": "Miller", "age": 29, "address": "222 Robomaker ave, New Yotk 13579"},
  {"id": 5, "first_name": "Mark", "last_name": "Davis", "age": 31, "address": "444 Transcribe blvd, Florida 02468"},
  {"id": 6, "first_name": "Laura", "last_name": "Wilson", "age": 24, "address": "555 CodeGuru st, California 98765" },
  {"id": 7, "first_name": "Steve", "last_name": "Moore", "age": 36, "address": "456 DeepLens st, Texas 11223"},
  {"id": 8, "first_name": "Michelle", "last_name": "Chen", "age": 22, "address": "642 DeepCompose st, Colorado 33215"},
  {"id": 9, "first_name": "David", "last_name": "Lee", "age": 29, "address": "777 S3 st, California 99567"},
  {"id": 10, "first_name": "Jessica", "last_name": "Brown", "age": 18, "address": "909 Ec st, Utah 43210"}]

def address_lookup(id):
    for customer in customer_table:
        if customer["id"] == int(id):
            return customer

    return None

print(address_lookup(customer_id)["address"])